In [0]:
# Databricks notebook source
from pyspark.sql import SparkSession

# Read the encounter data for both hospitals from the bronze layer
hosa_encounters_df = spark.read.parquet("/mnt/rcmabhi/bronze/hosa/encounters")
hosb_encounters_df = spark.read.parquet("/mnt/rcmabhi/bronze/hosb/encounters")

# Union the two DataFrames into a single one. 
# unionByName is used as a best practice to avoid issues if column order differs.
encounters_df = hosa_encounters_df.unionByName(hosb_encounters_df)

# Create a temporary view so it can be used in the following SQL commands
encounters_df.createOrReplaceTempView("encounters")

# Display the combined data to verify
display(encounters_df)

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW quality_checks AS
SELECT 
  concat(EncounterID,'-',datasource) as EncounterID,
  EncounterID as SRC_EncounterID,
  PatientID,
  cast(EncounterDate as date) as EncounterDate,
  EncounterType,
  ProviderID,
  DepartmentID,
  ProcedureCode,
  cast(InsertedDate as date) as SRC_InsertedDate,
  cast(ModifiedDate as date) as SRC_ModifiedDate,
  datasource,
  CASE 
    WHEN EncounterID IS NULL OR PatientID IS NULL THEN TRUE
    ELSE FALSE
  END AS is_quarantined
FROM encounters

In [0]:
%sql
CREATE TABLE IF NOT EXISTS silver.encounters (
  EncounterID string,
  SRC_EncounterID string,
  PatientID string,
  EncounterDate date,
  EncounterType string,
  ProviderID string,
  DepartmentID string,
  ProcedureCode integer,
  SRC_InsertedDate date,
  SRC_ModifiedDate date,
  datasource string,
  is_quarantined boolean,
  audit_insertdate timestamp,
  audit_modifieddate timestamp,
  is_current boolean
)
USING DELTA;

In [0]:
%sql
-- Update old record to implement SCD Type 2
MERGE INTO silver.encounters AS target
USING quality_checks AS source
ON target.EncounterID = source.EncounterID AND target.is_current = true
WHEN MATCHED AND (
    target.SRC_EncounterID != source.SRC_EncounterID OR
    target.PatientID != source.PatientID OR
    target.EncounterDate != source.EncounterDate OR
    target.EncounterType != source.EncounterType OR
    target.ProviderID != source.ProviderID OR
    target.DepartmentID != source.DepartmentID OR
    target.ProcedureCode != source.ProcedureCode OR
    target.SRC_InsertedDate != source.SRC_InsertedDate OR
    target.SRC_ModifiedDate != source.SRC_ModifiedDate OR
    target.datasource != source.datasource OR
    target.is_quarantined != source.is_quarantined
) THEN
  UPDATE SET
    target.is_current = false,
    target.audit_modifieddate = current_timestamp()

In [0]:
%sql
-- Insert new record to implement SCD Type 2
MERGE INTO silver.encounters AS target 
USING quality_checks AS source 
ON target.EncounterID = source.EncounterID AND target.is_current = true
WHEN NOT MATCHED THEN
  INSERT
    (
      EncounterID,
      SRC_EncounterID,
      PatientID,
      EncounterDate,
      EncounterType,
      ProviderID,
      DepartmentID,
      ProcedureCode,
      SRC_InsertedDate,
      SRC_ModifiedDate,
      datasource,
      is_quarantined,
      audit_insertdate,
      audit_modifieddate,
      is_current
    )
  VALUES
    (
      source.EncounterID,
      source.SRC_EncounterID,
      source.PatientID,
      source.EncounterDate,
      source.EncounterType,
      source.ProviderID,
      source.DepartmentID,
      source.ProcedureCode,
      source.SRC_InsertedDate,
      source.SRC_ModifiedDate,
      source.datasource,
      source.is_quarantined,
      current_timestamp(),
      current_timestamp(),
      true
    );